<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/ice%26fire/Ice%26Fire_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install graphdatascience

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 183 kB 5.1 MB/s 
     |████████████████████████████████| 173 kB 38.6 MB/s 
  Created wheel for neo4j: filename=neo4j-5.2.0-py3-none-any.whl size=248021 sha256=7eca51544ff2688bb0223e2c4bad8b9d4f6ad129b0fa70670353498914a04c33
  Stored in directory: /root/.cache/pip/wheels/5a/07/16/4d845d69ef310660c14b7148848c95da3ef3950c7b58daec42
Successfully built neo4j


In [2]:
from graphdatascience import GraphDataScience

host = "bolt://44.202.221.209:7687"
user = "neo4j"
password = "map-striker-injuries"

gds = GraphDataScience(host, auth=(user, password))

In [3]:
# Results of the query changed for notebook environment
gds.run_cypher(
    """MATCH (s:Character {name:$person1}), (t:Character {name:$person2})
MATCH p=shortestPath((s)-[:FATHER|MOTHER|SPOUSE*]-(t))
RETURN [n in nodes(p) | n.name] AS result""",
    {"person1": "Tyrion Lannister", "person2": "Viserys I"},
)["result"][0]

['Tyrion Lannister',
 'LadyJoanna Lannister',
 'Cersei Lannister',
 'Robert I',
 'Steffon Baratheon',
 'Rhaelle Targaryen',
 'Aegon V',
 'Maekar I',
 'Daeron II',
 'Aegon IV',
 'Viserys II',
 'Rhaenyra Targaryen',
 'Viserys I']

In [4]:
# Results of the query changed for notebook environment
gds.run_cypher(
    """MATCH p=(c:Character {name:$person})-[:FATHER|MOTHER*]->()
RETURN [r in relationships(p) | endNode(r).name + " " + type(r) + " to " + startNode(r).name] AS result""",
    {"person": "Margaery Tyrell"},
)["result"]

0              [Mace Tyrell FATHER to Margaery Tyrell]
1    [Mace Tyrell FATHER to Margaery Tyrell, Olenna...
2    [Mace Tyrell FATHER to Margaery Tyrell, Olenna...
3    [Mace Tyrell FATHER to Margaery Tyrell, Luthor...
4         [Alerie Hightower MOTHER to Margaery Tyrell]
5    [Alerie Hightower MOTHER to Margaery Tyrell, L...
Name: result, dtype: object

In [5]:
G, res = gds.graph.project("family", "Character", ["MOTHER", "FATHER", "SPOUSE"])

In [6]:
wcc_df = gds.wcc.stream(G)
wcc_df["name"] = [el["name"] for el in gds.util.asNodes(wcc_df["nodeId"].to_list())]
wcc_df["last_name"] = [
    el.split(" ")[-1] if len(el.split(" ")) > 1 and len(el.split(" ")[-1]) > 3 else None
    for el in wcc_df["name"]
]

In [7]:
wcc_df.head()

,nodeId,componentId,name,last_name
0,4512,0,Arianne Martell,Martell
1,4513,1,Arianne Tarth,Tarth
2,4514,2,Arlan of Pennytree,Pennytree
3,4516,3,Arlan III Durrandon,Durrandon
4,4517,4,Arlan V Durrandon,Durrandon


In [8]:
wcc_df.groupby("componentId").size().sort_values(ascending=False).to_frame(
    "componentSize"
).reset_index().head()

,componentId,componentSize
0,4,785
1,235,19
2,295,12
3,726,11
4,448,10


In [11]:
largest_component = wcc_df.groupby('componentId').size().sort_values(
    ascending=False
).reset_index()['componentId'][0]
wcc_df[wcc_df["componentId"] == largest_component].groupby("last_name").size().sort_values(
    ascending=False
).to_frame("count").reset_index().head(10)

,last_name,count
0,Frey,91
1,Targaryen,66
2,Stark,50
3,Lannister,30
4,Hightower,28
5,Velaryon,21
6,Baratheon,21
7,Greyjoy,19
8,Rivers,15
9,Arryn,15


In [12]:
gds.run_cypher("""
MATCH (c1:Character)<-[:FATHER|MOTHER]-(s1)-[:SPOUSE]-(s2)-[:FATHER|MOTHER]->(c2:Character)
WHERE c1.name CONTAINS "Targaryen" AND c2.name CONTAINS "Stark"
RETURN s1.name AS spouse1, s2.name AS spouse2
""")

,spouse1,spouse2
0,Jacaerys Velaryon,Sara Snow


In [ ]:
G.drop()

graphName                                                       family
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                         3654
relationshipCount                                                 1794
configuration        {'relationshipProjection': {'FATHER': {'orient...
density                                                       0.000134
creationTime                       2022-11-08T17:57:36.072746000+00:00
modificationTime                   2022-11-08T17:57:36.181971000+00:00
schema               {'graphProperties': {}, 'relationships': {'FAT...
Name: 0, dtype: object